In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix_car


In [3]:
!pwd

/content/drive/My Drive/Colab Notebooks/dw_matrix_car


In [4]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance
import tensorflow
print(tensorflow.__version__)

import xgboost as xgb

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


1.15.0


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Featyre Enginering

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_value=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_value
  else:
    df[feat +SUFFIX_CAT]=factorized_value
  #print(feat)

In [7]:
cat_feats=[x for x in df.columns if SUFFIX_CAT in x]
cat_feats=[x for x in cat_feats if 'price' not in x]
cat_feats

['created_at__cat',
 'seller_address__cat',
 'seller_name__cat',
 'seller_type__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_poduszka-powietrzna-chroniąca-kolana__cat',
 'feature_kurtyny-powietrzne__cat',
 'feature_klimatyzacja-dwustrefowa__cat',
 'feature_światła-led__cat',
 'feature_czujnik-zmierzchu__cat',
 'feature_elektrycznie-ustawiane-lusterka__cat',
 'feature_asr-(kontrola-trakcji)__cat',
 'feature_poduszka-powietrzna-kierowcy__cat',
 'feature_cd__cat',
 'feature_elektryczne-szyby-przednie__cat',
 'feature_poduszka-powietrzna-pasażera__cat',
 'feature_system-start-stop__cat',
 'feature_światła-do-jazdy-dziennej__cat',
 'feature_komputer-pokładowy__cat',
 'feature_elektryczne-szyby-tylne__cat',
 'feature_klimatyzacja-manualna__cat',
 'feature_tapicerka-welurowa__cat',
 'feature_czujnik-deszczu__cat',
 'feature_światła-przeciwmgielne__cat',
 'feature_ogrzewanie-postojowe__cat',
 'feature_radio-niefabryczne__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_

In [0]:
df['param_rok-produkcji__cat']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))

In [0]:
df.param_moc__cat=df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(x.replace(' KM', '').replace(' ', '')))
df.param_przebieg__cat=df['param_przebieg'].map(lambda x: -1 if str(x)=='None' else int(str(x).split('km')[0].replace(' ', '')))
df['param_pojemność-skokowa__cat'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(str(x).split('cm')[0].replace(' ', '')))
def run_model(model, feats):
  X=df[feats].values
  y=df.price_value.values
  scores=cross_val_score(model, X, y, cv=10, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
feat_s=[
'param_rok-produkcji__cat',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'seller_name__cat',
'param_model-pojazdu__cat',
'feature_wspomaganie-kierownicy__cat',
'param_kod-silnika__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_system-start-stop__cat',
'param_przebieg__cat',
'param_color__cat',
'feature_asystent-pasa-ruchu__cat' ]

In [0]:
xgb_param={
    'max_depth':7, 
    'n_estimators':25, 
    'learning_rate':0.1,
    'seed':0
}
m=run_model(model=xgb.XGBRegressor(**xgb_param), feats=cat_feats)

# Hyperopt

In [19]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feat_s)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_params={
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31,0.05)),
    'max_depth':hp.choice('max_depth', np.arange(5, 16,1, dtype=int)),
    'subsample':hp.quniform('subsample', 0.5, 1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1,0.05),
    'n_estimators':25, 
    'objective':'reg:squarederror',
    'seed':0
}

## run
best=fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 25, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 6, 'n_estimators': 25, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 25, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.05, 'max_depth': 12, 'n_estimators': 25, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 6, 'n_estimators': 25, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.70000000

{'colsample_bytree': 0.8500000000000001,
 'learning_rate': 3,
 'max_depth': 8,
 'subsample': 1.0}